In [1]:
import sys
#!{sys.executable} -m pip install efficientnet
!{sys.executable} -m pip install keras
!{sys.executable} -m pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     |████████████████████████████████| 20.1 MB 24.7 MB/s            
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5


In [2]:
# Get pretrained model 
from tensorflow.keras.applications.efficientnet import EfficientNetB0,preprocess_input,decode_predictions
from tensorflow.keras.preprocessing import image

# Get Connection
import boto3
from boto3 import Session

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Handle Images
from PIL import Image
from io import BytesIO
import io

session = Session()
credentials = session.get_credentials()

#aws access credentials
current_credentials = credentials.get_frozen_credentials()

client = boto3.client(
    's3',
    aws_access_key_id=current_credentials.access_key,
    aws_secret_access_key=current_credentials.secret_key,
    aws_session_token=current_credentials.token)

Matplotlib is building the font cache; this may take a moment.
/home/ec2-user/anaconda3/envs/amazonei_tensorflow2_p36/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


In [3]:
bucket_name = 'tellisa-collage-gan'
prefix = 'collage_gan/train/'

In [4]:
paginator = client.get_paginator('list_objects_v2')
pages = paginator.paginate(Bucket=bucket_name,Prefix=prefix) 

objKeyFrame = pd.DataFrame()
for page in pages:
    objKeyFrame = pd.concat([objKeyFrame,pd.DataFrame(page['Contents'])],ignore_index=True )

In [5]:
objKeyFrame = objKeyFrame[objKeyFrame['Size']>0]
objKeyFrame

,Key,LastModified,ETag,Size,StorageClass
1,collage_gan/train/_.lunatico.__pic#0.jpg,2022-11-23 00:57:03+00:00,"""77fb71617b0a87bb0135ee115e421335""",50221,STANDARD
2,collage_gan/train/_.lunatico.__pic#1.jpg,2022-11-23 00:57:04+00:00,"""b592242bc5a7ed6f8ce352ca956193c3""",111230,STANDARD
3,collage_gan/train/_.lunatico.__pic#10.jpg,2022-11-23 00:57:07+00:00,"""d21a9a6640498d7f16459f906e4a67d8""",74531,STANDARD
4,collage_gan/train/_.lunatico.__pic#100.jpg,2022-11-23 00:57:40+00:00,"""1e3500cd852fc8e25c352cd0b0f83dfe""",97581,STANDARD
5,collage_gan/train/_.lunatico.__pic#101.jpg,2022-11-23 00:57:41+00:00,"""c1ab15461ed4c089959ad52e099e33c7""",119971,STANDARD
...,...,...,...,...,...
20123,collage_gan/train/vertigo.artography_pic#93.jpg,2022-11-23 04:49:27+00:00,"""e67048069aa77b8a72b80cb5fc9d246d""",50659,STANDARD
20124,collage_gan/train/vertigo.artography_pic#95.jpg,2022-11-23 04:49:28+00:00,"""eecf8607cbd4ee5c2518d175710d534f""",67896,STANDARD
20125,collage_gan/train/vertigo.artography_pic#96.jpg,2022-11-23 04:49:28+00:00,"""1f84e5ceaba97f0f9a32e64dcf6b4476""",29229,STANDARD
20126,collage_gan/train/vertigo.artography_pic#97.jpg,2022-11-23 04:49:29+00:00,"""0bf8c83ed4229f34d2e73a4dceda88ed""",53165,STANDARD


In [6]:
import concurrent.futures 

def get_image(img_key):
    bucket_name = 'tellisa-collage-gan'
    img_inst = client.get_object(Bucket=bucket_name,Key=img_key)
    return img_inst['Body'].read()



In [7]:
imageMetadata = []
with concurrent.futures.ThreadPoolExecutor() as executor:
    results = executor.map(get_image,objKeyFrame.Key)
    for result in results:
        imageMetadata.append(result)

In [8]:
model = EfficientNetB0(include_top = False, weights = 'imagenet', pooling='avg')

def get_image_vector(img):
    img = Image.open(io.BytesIO(img))  #(img['Body'].read()))
    x = image.img_to_array(img)
    x = np.expand_dims(x,axis=0)
    x = preprocess_input(x)
    preds = model.predict(x)
    return pd.DataFrame(preds[0]).T

16711680/16705208 [==============================] - 0s 0us/step


In [9]:
import time

imageVectorFrame = pd.DataFrame()
numIters = 0 
for i in imageMetadata:
    imageVectorFrame = pd.concat([imageVectorFrame,get_image_vector(i)],ignore_index=True)
    if numIters%1000==0:
        print(numIters,time.time())
    numIters+=1

0 1674402704.1412551
1000 1674402943.0476825
2000 1674403182.3387535
3000 1674403409.9136512
4000 1674403651.679424
5000 1674403865.0062187
6000 1674404066.7583787
7000 1674404301.6195574
8000 1674404559.059302
9000 1674404821.0692544
10000 1674405043.1658895
11000 1674405258.9062057
12000 1674405473.6161861
13000 1674405698.6463513
14000 1674405988.0696218
15000 1674406246.6796136
16000 1674406491.4943259
17000 1674406759.2871308
18000 1674407048.5564477
19000 1674407354.3027565
20000 1674407595.6462128


In [10]:
imageVectorFrame

,0,1,2,3,4,5,6,7,8,9,...,1270,1271,1272,1273,1274,1275,1276,1277,1278,1279
0,-0.073504,-0.081529,-0.150256,-0.039397,-0.196335,-0.058430,0.089128,-0.125144,-0.161661,-0.073682,...,-0.047790,-0.013145,0.001945,-0.036383,-0.208646,-0.152522,-0.067945,-0.050996,0.012269,-0.137268
1,-0.139749,0.065019,-0.103396,0.045213,0.049502,0.782018,0.078437,-0.067619,0.265398,0.007390,...,0.076177,0.134568,-0.130283,-0.071177,-0.042576,0.047607,0.232933,0.075863,0.197335,0.723809
2,0.045302,-0.000459,-0.135456,-0.122611,-0.127929,0.521024,-0.045943,-0.141464,0.125021,-0.034702,...,0.192965,-0.036452,-0.015031,-0.088858,0.292089,0.337222,0.057235,-0.113923,-0.064440,0.202605
3,-0.029537,0.258865,-0.110284,0.350379,0.003658,0.724655,0.097174,-0.080560,1.121670,0.201151,...,-0.080352,-0.034084,0.338343,0.185316,0.295396,0.007392,0.274381,0.205001,-0.007172,0.544511
4,0.391050,-0.089162,-0.087924,0.546654,-0.048115,-0.033974,0.539756,-0.120903,0.191909,-0.070164,...,0.073363,0.156908,-0.022547,0.106340,-0.017653,0.334020,-0.024949,-0.004910,0.153678,0.963530
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20122,-0.103502,-0.191089,0.980874,0.848847,-0.137837,0.085290,0.077178,-0.127047,0.139809,-0.108439,...,0.247494,0.312148,0.264130,0.528798,0.073671,-0.182989,-0.133471,-0.021138,0.000589,0.432421
20123,0.372672,-0.043630,0.100556,0.033871,-0.078832,0.489081,0.209889,-0.084278,0.174035,-0.114111,...,0.203081,0.117522,0.036368,0.306143,0.303440,0.098530,0.061106,0.052438,0.341346,0.223040
20124,0.223605,0.408574,0.045650,-0.110085,0.667419,0.354027,0.375549,0.109929,0.022478,0.263314,...,0.345008,-0.040833,0.041099,-0.168007,1.019260,-0.162075,0.083081,1.821774,1.152534,0.478172
20125,0.818553,0.074903,-0.109544,0.124542,0.082014,-0.060304,0.539175,-0.099053,0.466950,0.178996,...,-0.069705,0.253560,-0.081677,0.209007,0.070019,0.112666,-0.053768,-0.054625,0.067755,1.063112


In [11]:
imageVectorFrame.to_csv('collage_vectors.csv')
objKeyFrame.to_csv('collage_s3keys.csv')